# Convolutional Neural Network tutorial

This is an example of how neural networks can be used to categorize images.  The test images are the MNIST collection of handwritten digits, and we will train a network to recognized hand-written digits.  Other sets of data are available.

### Import packages and dataset
Alternative datasets are cifar10 and cifar100.

In [ ]:
%matplotlib inline
from tensorflow.keras.datasets import mnist, cifar10, cifar100
from tensorflow.keras import layers
import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt

### Look at the dataset
Data scientists always start by visualizing their data - avoid surprises later.

In [ ]:
#data = mnist.load_data()   # MNIST will not work with this network - missing color
                            # channel, dimensions not divisible by 4.
data = cifar10.load_data()
#data = cifar100.load_data()

(train_images, train_labels), (test_images, test_labels) = data
ntrain = len(train_images)
imgsize = train_images.shape[1:]
print("Training set: {} images of shape {}".format(ntrain, str(imgsize)))
print("Labels have shape", train_labels.shape)
num_categories = train_labels.max() + 1
print("Number of categories:", num_categories)

# For CIFAR data the shape is (N, 1), change to (N,)
if len(train_labels.shape) == 2:
    assert train_labels.shape[-1] == 1
    train_labels.shape = (-1,)
    test_labels.shape = (-1,)

For CICAR-10 data, the categories can be found at https://www.cs.toronto.edu/~kriz/cifar.html

In [ ]:
nplot = 10
rnd = np.random.randint(ntrain, size=(nplot,))
fig, axes = plt.subplots(1, nplot, figsize=(12,12/nplot))
for i in range(nplot):
    axes[i].imshow(train_images[rnd[i]], cmap='gray')
    axes[i].axis('off')
print(train_labels[rnd])

The neural network expects floating point numbers of order unity.  The image data comes as integers (probably from 0 to 255), we convert it to floats in the range 0.0 - 1.0.

In [ ]:
print(train_images.min(), train_images.max())
maxval = train_images.max()
train_images = train_images.astype(np.float32) / maxval
test_images = test_images.astype(np.float32) / maxval

### Define a function creating the neural network
Make multiple functions to be able to experiment with multiple architectures.

Note:
* Useful values for ``activation``: 'sigmoid', 'tanh', 'relu', None
* Useful values for ``padding``: 'valid', 'same'


In [ ]:
def make_minimalnet(categories, input_shape, hiddenneurons=200):
    network = keras.models.Sequential()
    # The first layer needs to know the size of the input.
    # The following layers infer their size from the previous layer.
    network.add(layers.Conv2D(32, 5, input_shape=input_shape,
               activation='sigmoid', padding='same'))
    network.add(layers.MaxPooling2D(pool_size=2))
    network.add(layers.Conv2D(32, 5,
               activation='sigmoid', padding='same'))
    network.add(layers.Flatten())
    network.add(layers.Dense(hiddenneurons, activation='sigmoid'))
    network.add(layers.Dense(categories, activation='softmax'))
    return network

### Train the network

In [ ]:
net = make_minimalnet(num_categories, imgsize)
net.summary()

In [ ]:
net.compile(optimizer='rmsprop',
            loss='categorical_crossentropy',
            metrics=['accuracy'])
# The labels should be one-hot encoded.
train_labels_onehot = keras.utils.to_categorical(train_labels)

In [ ]:
net.fit(train_images, train_labels_onehot, epochs=20, batch_size=128)

### Test the network on the testset.

We let Keras run the network on the test set, and evaluate the results

In [ ]:
test_labels_onehot = keras.utils.to_categorical(test_labels)
test_loss, test_acc = net.evaluate(test_images, test_labels_onehot)
print('Test set accuracy:', test_acc)


### Show some of the failures

We run the network on the test data again, this time keeping the results and looking at them

In [ ]:
results =  net.predict(test_images)

The output is a matrix of probabilities.  Each row contains the probabilities that the image belongs to each of the ten classes.  Let us look at the first five

In [ ]:
with np.printoptions(precision=4, suppress=True):
    print(results[:5])

Convert this into predictions: find the most probable class for each image.

In [ ]:
predicted_class = np.argmax(results, axis=1)
predicted_class.shape

In [ ]:
n_error = (test_labels != predicted_class).sum()
print("There are {} errors.".format(n_error))

Find five failures.  Show them.

In [ ]:
failures = np.argsort(test_labels != predicted_class)[-n_error:]

In [ ]:
nshow = 5
fig, axes = plt.subplots(1, nshow, figsize=(12,12/nplot))

for i, f in enumerate(failures[:nshow]):
    axes[i].imshow(test_images[f],cmap='gray')
    axes[i].axis('off')
    print("{}:  Predicted: {}  Correct class: {}".format(
        f, predicted_class[f], test_labels[f]))